## Astra Schema

### Initialise Connection

In [2]:
from astra_db import AstraSession

session = AstraSession().session

## Schema

### Create Tables

In [3]:
#
# Communications Table
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS communications (
    communication_id TEXT,
    comm_date TIMESTAMP,
    customer_id TEXT,
    channel TEXT,
    category_group TEXT,
    category TEXT,
    activity_name TEXT,
    PRIMARY KEY ((communication_id), comm_date) );
    """
)

### Create CAP Tables

In [5]:
#
# Communications by Activity Day
#
# This table will store the number of messages sent in a day for a given activity.
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS comm_by_activity_day (
        activity_name text,
        comm_date_bucket int,
        comm_time_bucket int,
        comm_offset_time int static,
        comm_offset_count int static,
        PRIMARY KEY ((activity_name, comm_date_bucket), comm_time_bucket)
    ) WITH CLUSTERING ORDER BY (comm_time_bucket DESC) AND default_time_to_live = 604800;
    """
)

In [ ]:
#
# PROFILECAP - customer_id, channel
# # 20250205
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS profile_cap (
    communication_id TEXT,
    customer_id TEXT,
    channel TEXT,
    comm_date_bucket INT,
    PRIMARY KEY ((customer_id, channel, comm_date_bucket), communication_id) );
    """
)

In [ ]:
#
# PRTYCAP - category_group, category
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS prty_cap (
    communication_id TEXT,
    category_group TEXT,
    category TEXT,
    comm_date_bucket INT,
    PRIMARY KEY ((category_group, category, comm_date_bucket), communication_id) );
    """
)

In [ ]:
#
# CHANNELCAP - channel
#
session.execute(
    """
    CREATE TABLE IF NOT EXISTS channel_cap (
    communication_id TEXT,
    channel TEXT,
    comm_date_bucket INT,
    PRIMARY KEY ((channel, comm_date_bucket), communication_id) );
    """
)

## Drop Schema

In [ ]:
session.execute("DROP TABLE IF EXISTS communications")
session.execute("DROP TABLE IF EXISTS profile_cap")
session.execute("DROP TABLE IF EXISTS acty_cap")
session.execute("DROP TABLE IF EXISTS prty_cap")
session.execute("DROP TABLE IF EXISTS channel_cap")